# Deep Learning from a Software Engineer's Perspective
This notebook explains the process of building and training a deep learning model with PyTorch. The steps include loading and validating data, creating a neural network, training, and finally making predictions.

## Problem Identification
The task is to classify images into two categories: 'cat' and 'fish'. We'll start by processing our data and training a neural network to solve this problem.

## Data Processing
We will load and validate images to ensure there are no corrupted files.

In [ ]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image, UnidentifiedImageError
import os

# Define transformations for the data
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64 (width x height)
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Define the data paths
train_data_path = "./images/train/"
val_data_path = "./images/val/"
test_data_path = "./images/test/"

def validate_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            try:
                img = Image.open(os.path.join(root, file))
                img.verify()  # Check if it's a valid image
            except (IOError, UnidentifiedImageError):
                print(f"Removing corrupted image: {file}")
                os.remove(os.path.join(root, file))  # Remove corrupted file

validate_images(train_data_path)
validate_images(val_data_path)
validate_images(test_data_path)

## Loading the Data
Next, we load the dataset and apply the transformations defined above.

In [ ]:
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=data_transforms)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=data_transforms)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=data_transforms)

batch_size = 64

train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

## Neural Network
We'll start with a simple feedforward neural network (NeuralNet) and then switch to a more complex convolutional neural network (CNNNet).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Next, we define a more complex convolutional neural network for better accuracy.

In [ ]:
class CNNNet(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=3, stride=2),

                                      nn.Conv2d(64, 192, kernel_size=5, padding=2),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=3, stride=2),

                                      nn.Conv2d(192, 384, kernel_size=3, padding=1),
                                      nn.ReLU(),
                                      nn.Conv2d(384, 256, kernel_size=3, padding=1),
                                      nn.ReLU(),
                                      nn.Conv2d(256, 256, kernel_size=3, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=3, stride=2))
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(nn.Dropout(),
                                        nn.Linear(256 * 6 * 6, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Linear(4096, num_classes))
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## Training the Model
We'll train the model using Adam optimizer and cross-entropy loss. The training loop will run for 20 epochs.

In [ ]:
import torch.optim as optim
import torch

model = CNNNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    model.to(device)

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            
            training_loss += loss.item()
        
        training_loss /= len(train_loader)
        
        model.eval()
        num_correct = 0
        num_examples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                output = model(inputs)
                loss = loss_fn(output, targets)
                valid_loss += loss.item()
                
                correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
                num_correct += torch.sum(correct).item()
                num_examples += correct.shape[0]
        
        valid_loss /= len(val_loader)
        accuracy = num_correct / num_examples
        
        print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {training_loss:.2f}, Validation Loss: {valid_loss:.2f}, Accuracy: {accuracy:.2f}')
        
train(model, optimizer, loss_fn, train_data_loader, val_data_loader)

## Saving the Model
After training, the model is saved for future use.

In [ ]:
torch.save(model, "./model/simple_model")

## Making Predictions
We will now load the saved model and use it to make predictions on a new image.

In [ ]:
from PIL import Image

model = torch.load("./model/simple_model")
labels = ['cat', 'fish']
img = Image.open("pexels-crisdip-35358-128756.jpg")
img = data_transforms(img)
img = img.unsqueeze(0)
prediction = model(img)
prediction = prediction.argmax()
print(labels[prediction])